### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [4]:
# Write your code from here
import sys
import os
sys.path.append('.')
import grpc
import time
from concurrent import futures
import re
import validator_pb2
import validator_pb2_grpc
class ValidatorServiceServicer(validator_pb2_grpc.ValidatorServiceServicer):
    def ValidateUser(self,request,context):
        user_data=request.user_data
        is_valid=True
        error_messages=[]
        if user_data.user_id<=0:
            is_valid=False
            error_messages.append("User ID must be positive.")
        if not user_data.name:
            is_valid=False
            error_messages.append("Name cannot be empty.")
        if not re.match(r"[^@]+@[^@]+\.[^@]+",user_data.email):
            is_valid=False
            error_messages.append("Invalid email format.")
        if user_data.age<0 or user_data.age>120:
            is_valid=False
            error_messages.append("Age must be between 0 and 120.")
        return validator_pb2.ValidationResponse(is_valid=is_valid,error_messages=error_messages)
def serve():
    server=grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validator_pb2_grpc.add_ValidatorServiceServicer_to_server(
        ValidatorServiceServicer(),server)
    server.add_insecure_port('[::]:50051')
    server.start()
    print("Serverstarted,listeningonport50051.")
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        server.stop(0)
if __name__=='__main__':
    serve()

ModuleNotFoundError: No module named 'validator_pb2'